In [ ]:
#extract response amplitudes from each trial
import os
import mne
import numpy as np
from reftep_util_funcs import *
import shutil
responses_dipoles = ['n15','p30','n45','p60']
responses_emg = ['mep']
responses_all = responses_dipoles + responses_emg



for site in ['Tuebingen','Aalto']:
    site_path_sources = fr"D:\REFTEP_ALL\Source_analysis\Source_analysis_{site}"
    site_path_preprocessing = fr"D:\REFTEP_ALL\EEG_preprocessing_data\Preprocessing_{site}"
    for subject in os.listdir(site_path_sources):
        subject_source_path = os.path.join(site_path_sources,subject)
        forward = mne.read_forward_solution(os.path.join(subject_source_path,f'{subject}-fwd.fif'),verbose=False) #read forward solution
        #read eeg epochs
        epochs = mne.read_epochs(os.path.join(subject_source_path,f'{subject}_final_eeg-epo.fif'),verbose=False)
        epochs_data = epochs.get_data(copy=True) #load the epochs data into a separate structure
        n_trials = epochs_data.shape[0] #number of trials
        #read emg epochs
        emg_epochs = mne.read_epochs_eeglab(os.path.join(site_path_preprocessing,subject,f'{subject}_EMG_aligned_final.set'),verbose=False) #load emg data
        #define paths to save data to
        dipoles_folder_path = os.path.join(subject_source_path,f'{subject}_dipoles')
        mep_ptps_filepath = os.path.join(subject_source_path,f'{subject}_mep_amplitudes')
        for response_type in responses_all:
            if response_type in responses_emg:
                mep_ptps = get_mep_ptps(emg_epochs,tmin=0.02, tmax=0.05)
                np.save(mep_ptps_filepath, mep_ptps)
                print(f'{subject}, mep amps mean std, {np.mean(mep_ptps)}, {np.std(mep_ptps)}')
            elif response_type in responses_dipoles: #dipoles separately to emg
                dipole_now = mne.read_dipole(os.path.join(dipoles_folder_path,f'{subject}_dipole_{response_type}')) #filepath for the dipole
                potential_times = np.load(os.path.join(dipoles_folder_path,f'{subject}_dipole_{response_type}_fitting_times.npy'))
                tmin, tmax = np.min(potential_times), np.max(potential_times) #min and max time of the fitting range
                pos_ind = int(dipole_now.name.split("_")[-1]) #the position of the dipole is saved here
                dipoles_path_now = os.path.join(dipoles_folder_path,f'{subject}_dipole_{response_type}_trials') #define path to save single trial dipoles
                if os.path.exists(dipoles_path_now):
                    shutil.rmtree(dipoles_path_now)
                os.makedirs(dipoles_path_now,exist_ok=True)
                dipamps = []
                dipgofs = []
                for trial_ind in range(n_trials):
                    evoked_trial = mne.EvokedArray(data=epochs_data[trial_ind,:,:], info=epochs.info, tmin=epochs.times[0],nave=1, kind='single_epoch',verbose=False)
                    dipole_trial, _, _ = lsq_dipole_to_pos(forward, evoked_trial, tmin, tmax, pos_ind, n_times=int(round((tmax-tmin)*1000))+1, ori_fixed=None) #use free orientation to extract responses
                    dipgofs.append(dipole_trial.gof[0])
                    dipamps.append(dipole_trial.amplitude[0])
                    trial_dipole_path = os.path.join(dipoles_path_now,f'{subject}_dipole_{response_type}_trial_{trial_ind}')
                    dipole_trial.save(trial_dipole_path,overwrite=True,verbose=False)
                print(f'{subject},{response_type}, mean and std of gof and amp {np.mean(dipgofs)}, {np.std(dipgofs)}, {np.mean(dipamps)*1e9}, {np.std(dipamps)*1e9}')
